<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/visual_coherence_without_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Mounted at /content/gdrive


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

100% 2.54G/2.55G [00:25<00:00, 100MB/s] 
100% 2.55G/2.55G [00:25<00:00, 109MB/s]
--2020-12-08 14:04:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-08 14:04:32--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-08 14:04:32--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200

In [14]:
def train(model, iter):
  for Xs, Ys in iter:
    with tf.GradientTape() as tape:
      #print("size of iter: ",len(iter))
      logits = model(Xs)
      loss = model.loss(logits, Ys)
      print("train, losse:",loss)
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

def test(model, iter):
  n = 0
  m = 0
  for Xs, Ys in iter:
    print("-----------------------")
    print("testing, Xs:",len(Xs))
    print("testing, n: ",n)
    n += len(Xs)
    probs = model(Xs)
    m += sum(np.argmax(probs,-1)==Ys)
  print("final test accuracy:",m/n)
  return m/n

model = Model(l_embed)
train(model, train_iter3)
test(model, test_iter3)

[[[<tf.Tensor: shape=(), dtype=int32, numpy=28636>, <tf.Tensor: shape=(), dtype=int32, numpy=21903>, <tf.Tensor: shape=(), dtype=int32, numpy=15671>, <tf.Tensor: shape=(), dtype=int32, numpy=28636>, <tf.Tensor: shape=(), dtype=int32, numpy=20497>, <tf.Tensor: shape=(), dtype=int32, numpy=28636>, <tf.Tensor: shape=(), dtype=int32, numpy=10463>, <tf.Tensor: shape=(), dtype=int32, numpy=24093>, <tf.Tensor: shape=(), dtype=int32, numpy=1639>, <tf.Tensor: shape=(), dtype=int32, numpy=6093>, <tf.Tensor: shape=(), dtype=int32, numpy=7675>, <tf.Tensor: shape=(), dtype=int32, numpy=21124>, <tf.Tensor: shape=(), dtype=int32, numpy=6769>, <tf.Tensor: shape=(), dtype=int32, numpy=28636>, <tf.Tensor: shape=(), dtype=int32, numpy=27127>, <tf.Tensor: shape=(), dtype=int32, numpy=12316>, <tf.Tensor: shape=(), dtype=int32, numpy=10423>, <tf.Tensor: shape=(), dtype=int32, numpy=28636>, <tf.Tensor: shape=(), dtype=int32, numpy=12112>, <tf.Tensor: shape=(), dtype=int32, numpy=0>, <tf.Tensor: shape=(), dty

InvalidArgumentError: ignored

2
recipe_id
context
choice_list
image_list
{'body': [18780, 19167, 20099, 5498, 25399, 11969, 26419, 28471, 27387, 1782, 14172, 2032, 20099, 9584, 25399, 29211], 'title': [537, 5498, 11300, 13779]}
{'body': [23447, 5895, 12979, 8138, 22562, 23447, 12250, 28471, 4599, 4810, 10799, 13779, 21626, 4841, 26280, 28471, 4841, 24895, 7398, 18035, 20099, 20520, 13659, 28471, 14230, 27203, 10396, 25762, 11654, 25829, 12979, 15736, 9153, 22232, 12979, 29325, 25236], 'title': [18180, 3722, 11537, 13779]}
{'body': [12413, 20099, 20909, 24751, 4599, 20867, 16172, 25399, 19013, 18807, 15636, 24533, 25399, 11969, 25945, 1782, 24533, 24089, 13771, 10396, 13547, 12979, 15002, 18294, 1896, 18291, 25399, 20099, 9584, 25399, 11094, 4923, 25945, 20099, 14386, 22210, 28471, 20790, 27199, 3001, 3396, 8640, 15668, 20099, 9584, 22969, 20099, 5498, 26479, 10396, 28811, 27199, 13254, 20099, 20909, 22969, 25510, 12557, 25694, 20867, 24551, 21053, 14324, 24999, 25716, 20099, 9293, 1702], 'title': [537, 20099, 9584,

In [68]:
class Model2(tf.keras.Model):
  def __init__(self, l_embed):
    super(Model2, self).__init__()
    self.window_sz = 500
    self.embedding_size = 100
    self.image_conv1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides=(2, 2), padding='valid')
    self.image_conv2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides=(2, 2), padding='valid')
    self.image_conv3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides=(2, 2), padding='valid')
    self.image_dense = tf.keras.layers.Dense(units = 500)

    # self.word_embedding = l_embed
    # self.text_step_conv1 = tf.keras.layers.Conv1D(filters = 16, kernel_size = 5)
    # self.text_step_conv2 = tf.keras.layers.Conv1D(filters = 32, kernel_size = 5)
    # self.text_step_conv3 = tf.keras.layers.Conv1D(filters = 32, kernel_size = 5)
    # self.text_step_maxpool = tf.keras.layers.MaxPool1D(pool_size = 3)
    # self.text_step_dense = tf.keras.layers.Dense(units = 100)

    # self.gru = tf.keras.layers.GRU(200, return_sequences=True, return_state=True)

    self.text_image_embedding1 = tf.keras.layers.Dense(units = 200, activation = "relu")
    self.text_image_embedding2 = tf.keras.layers.Dense(units = 100)

    self.class_dense1 = tf.keras.layers.Dense(units = 500, activation = "relu")
    self.class_dense2 = tf.keras.layers.Dense(units = 500, activation = "relu")
    self.class_dense3 = tf.keras.layers.Dense(units = 4)
    self.optimizer = tf.keras.optimizers.Adam(1e-3)

  def call(self, Xs):
    # textlist = []
    choicelist = []
    for recipe in Xs:#per data
      # text = []
      # step_list = []#all step per data
      # for step in recipe['context']:#per step
      #   step_list.append(step['body'])
      #   #text += step['body']
      # #print("call text len: ",len(text))
      # #print("call text: ",text)
      # #text = tf.reduce_mean([l_embed(tf.convert_to_tensor(item)) for item in text],axis=0)
      # textlist.append(step_list)
      choicelist.append(recipe['choice_list'])
    # textlist = tf.convert_to_tensor(textlist)
    # print("call textlist shape: ", textlist.shape)
    choice_image = tf.convert_to_tensor(choicelist)

    choice_token = self.image_conv1(choice_image[:,0])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice0_embedding = (tf.reshape(choice_token,(choice_token.shape[0],-1)))

    choice_token = self.image_conv1(choice_image[:,1])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice1_embedding = (tf.reshape(choice_token,(choice_token.shape[0],-1)))

    choice_token = self.image_conv1(choice_image[:,2])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice2_embedding = (tf.reshape(choice_token,(choice_token.shape[0],-1)))

    choice_token = self.image_conv1(choice_image[:,3])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice3_embedding = (tf.reshape(choice_token,(choice_token.shape[0],-1)))

    #query word embedding

    #text_embedding = self.text_dense(textlist)
    # text_embedding = []
    # for all_steps in textlist:#per data
    #     all_steps_embedding = []
    #     for step in all_steps:#per step
    #         step_token = self.text_step_conv1(step)
    #         step_token = self.text_step_conv2(step_token)
    #         step_token = self.text_step_conv3(step_token)
    #         step_token = self.text_step_maxpool(step_token)
    #         step_token = self.text_step_dense(step_token)
    #         all_steps_embedding.append(step_token)
    #     all_steps_embedding = tf.convert_to_tensor(all_steps_embedding)
    #     output, state = self.gru(all_steps_embedding)
    #     text_embedding.append(output)

    # text_embedding = tf.convert_to_tensor(text_embedding)

    # print("model.call, textlist shape: ",textlist.shape)
    # print("call, text_embedding shape: ",text_embedding.shape)
    # print("------------------------------")

    # #create image_word embedding for choice 0
    # token = tf.concat([choice0_embedding, text_embedding],axis=-1)
    # token = self.text_image_embedding1(token)
    # choice0_embedding = self.text_image_embedding2(token)

    #  #create image_word embedding for choice 1
    # token = tf.concat([choice1_embedding, text_embedding],axis=-1)
    # token = self.text_image_embedding1(token)
    # choice1_embedding = self.text_image_embedding2(token)

    #  #create image_word embedding for choice 2
    # token = tf.concat([choice2_embedding, text_embedding],axis=-1)
    # token = self.text_image_embedding1(token)
    # choice2_embedding = self.text_image_embedding2(token)

    #  #create image_word embedding for choice 3
    # token = tf.concat([choice3_embedding, text_embedding],axis=-1)
    # token = self.text_image_embedding1(token)
    # choice3_embedding = self.text_image_embedding2(token)

    token = tf.concat([choice0_embedding, choice1_embedding, choice2_embedding, choice3_embedding],axis=-1)
    token = self.class_dense1(token)
    token = self.class_dense2(token)
    logit = self.class_dense3(token)

    return logit

  def loss(self, logits, labels):
    # print("labels: ",labels)
    # print("logits: ",logits)
    return tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits))


In [63]:
batch_size = 32
(train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)


In [66]:
model2 = Model2(l_embed)

In [ ]:

for i in range(10): 
  (train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)
  train(model2, train_iter3)
  test(model2, test_iter3)

train, losse: tf.Tensor(35.49229, shape=(), dtype=float32)
train, losse: tf.Tensor(36.538612, shape=(), dtype=float32)
train, losse: tf.Tensor(36.70919, shape=(), dtype=float32)
train, losse: tf.Tensor(44.61967, shape=(), dtype=float32)
train, losse: tf.Tensor(38.135475, shape=(), dtype=float32)
train, losse: tf.Tensor(31.800552, shape=(), dtype=float32)
train, losse: tf.Tensor(39.21614, shape=(), dtype=float32)
train, losse: tf.Tensor(40.865997, shape=(), dtype=float32)
train, losse: tf.Tensor(38.62401, shape=(), dtype=float32)
train, losse: tf.Tensor(31.643154, shape=(), dtype=float32)
train, losse: tf.Tensor(34.257103, shape=(), dtype=float32)
train, losse: tf.Tensor(37.049614, shape=(), dtype=float32)
train, losse: tf.Tensor(39.248577, shape=(), dtype=float32)
train, losse: tf.Tensor(35.5746, shape=(), dtype=float32)
train, losse: tf.Tensor(40.21204, shape=(), dtype=float32)
train, losse: tf.Tensor(40.988495, shape=(), dtype=float32)
train, losse: tf.Tensor(40.3138, shape=(), dtype